In [ ]:
import numpy as np
import librosa
import librosa.display
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder

# Function to extract Mel spectrograms from audio files
def extract_mel_spectrogram(audio_file, n_mels=32, hop_length=256):
    y, sr = librosa.load(audio_file, sr=None)
    mel_spectrogram = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels, hop_length=hop_length)
    mel_spectrogram = librosa.power_to_db(mel_spectrogram, ref=np.max)
    return mel_spectrogram

# Load the trained model
model = load_model("mlp_model.h5")

# Load the label encoder
label_encoder = LabelEncoder()
label_encoder.classes_ = np.load("label_encoder.npy")

with open('max_width.txt', 'r') as file:
    max_width = int(file.read().strip())

# Function to predict bird species from audio file
def predict_bird_species(audio_file):
    # Extract Mel spectrogram from the audio file
    mel_spec = extract_mel_spectrogram(audio_file)
    
    # Pad the spectrogram to match the input shape of the model
    pad_width = max_width - mel_spec.shape[1]
    padded_spec = np.pad(mel_spec, ((0, 0), (0, pad_width)), mode='constant')
    
    # Reshape the spectrogram to add the channels dimension
    input_data = np.expand_dims(padded_spec, axis=0)
    
    # Make prediction
    prediction = model.predict(input_data)
    
    # Decode the prediction
    predicted_class = label_encoder.inverse_transform(np.argmax(prediction, axis=1))[0]
    
    return predicted_class

# Example usage
audio_file_path = "birds_audio_dataset\FIERY MINIVET\XC62167 - Fiery Minivet - Pericrocotus igneus igneus.mp3"
predicted_species = predict_bird_species(audio_file_path)
print("Predicted bird species:", predicted_species)
